Import tools.

In [1]:
import pandas as pd
import numpy as np
from pickle import load
from pickle import dump
pd.set_option("max_columns", None)
pd.set_option('max_rows', None)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import accuracy_score, make_scorer, f1_score, precision_score, recall_score

Bring in dataframe that has race, poverty, and results joined and has Alaska all tidied up.

In [2]:
df = load(open('race_poverty_results.pkl', 'rb'))

Look at and drop county and all but one state column for modeling. Drop id.

In [3]:
df = df.drop(['County_x', 'id', 'State_x', 'County_y', 'State_y', 'county'], axis = 1)

In [4]:
state_dummies = pd.get_dummies(df['state'], drop_first = True)
central_outlying = pd.get_dummies(df['central_outlying'], drop_first = True)

In [5]:
df = df.drop(['state', 'central_outlying'], axis = 1)

In [6]:
df = pd.concat([df, state_dummies, central_outlying], axis = 1)

In [7]:
df.Target = df.Target.map({'Trump': 0, 'Clinton': 1})

In [8]:
X = df.drop(['Target'], axis = 1)
y = df.Target

In [9]:
f1_score = make_scorer(f1_score)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 42)

In [11]:
param = {'svc__kernel': ['rbf', 'poly', 'linear']}
pipe2 = make_pipeline(StandardScaler(), svm.SVC())
grid = GridSearchCV(pipe2, param, scoring= f1_score)

In [12]:
grid.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'svc__kernel': ['rbf', 'poly', 'linear']},
             scoring=make_scorer(f1_score))

In [13]:
grid.best_score_

0.6842431244687643

In [14]:
grid.best_params_

{'svc__kernel': 'linear'}

In [15]:
grid.best_estimator_

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(kernel='linear'))])

In [16]:
y_hat_train = cross_val_predict(pipe2, X_train, y_train)

In [17]:
confusion_matrix(y_train, y_hat_train)

array([[1938,   40],
       [ 250,  127]], dtype=int64)